# convert_raw

This notebook applies various processing routines and conversions to the raw FlowMow2 data to generate L1B data.

#### Get UTM northing and easting values for nav

In [ ]:
import pandas as pd
nav = pd.read_hdf('../data/interim/nav_raw.h5', 'table')

In [ ]:
from pyproj import Proj
p = Proj(proj='utm', zone=9)

In [ ]:
nav['x'], nav['y'] = p(nav.lon.values, nav.lat.values)

#### Build a rotated coordinate system
Here we create a rotated coordinate system which may make gridding easier.

In [ ]:
import numpy as np
omega = 29 # rotate by omega degrees
theta = np.deg2rad(omega)
v = np.transpose(np.array([nav.x.values, nav.y.values]))
R = np.array([[np.cos(theta),-np.sin(theta)], [np.sin(theta),np.cos(theta)]])
vr = v.dot(R)
nav['xrot'] = vr[:,0]
nav['yrot'] = vr[:,1]

#### Save nav to hdf

In [ ]:
nav.to_hdf('../data/interim/nav.h5', 'table', append=False, data_columns=True)

#### Convert Paros values to temperature and pressure

Paros pressure values are in psia, which according to the [Paros manual](../docs/G8203_Digiquartz_Broadband_Pressure_Transducers_and_Depth_Sensors_with_Frequency_Outputs.pdf) can be converted to pascals by multipyling by 9806.650, or to m of H2O by multiplying by 0.7030696. Temperature is in degrees C. The SBE3 is likely the better instrument to use for temperature. The instument [calibration constants](../data/info/paros_cals.yaml) are extracted directly from the raw DAT files.

In [ ]:
paros = pd.read_hdf('../data/interim/paros_raw.h5', 'table')

In [ ]:
import yaml
import flowmow

In [ ]:
with open('../data/info/paros_cals.yaml') as f:
    paros_cals = yaml.load(f)

In [ ]:
paros['temp'], paros['pressure'] = flowmow.convert_paros(paros.eta, paros.tau, **paros_cals)

In [ ]:
# save to hdf5
#paros.to_hdf('../data/interim/paros.h5', 'table', append=False, data_columns=True)
#del paros

#### Convert SBE3 values to temperature

The Seabird SBE3 [calibration constants](../data/info/sbe3_cals.yaml) are from the [official calibration sheets](../docs/SBE03_2014_cals.pdf) which also contain the conversion equations. More info on the SBE3 [here](../docs/datasheet-03plus-May15.pdf).

In [ ]:
sbe3 = pd.read_hdf('../data/interim/sbe3_raw.h5', 'table')

In [ ]:
with open('../data/info/sbe3_cals.yaml') as f:
    sbe3_cals = yaml.load(f)

In [ ]:
# sbe3 2265 was on the stinger and was recorded as counts_0
sbe3['temp_stinger'] = flowmow.convert_sbe3(sbe3.counts_0, **sbe3_cals[2265])
sbe3['temp_top'] = flowmow.convert_sbe3(sbe3.counts_1, **sbe3_cals[2446])

In [ ]:
# save to hdf5
#sbe3.to_hdf('../data/interim/sbe3_proc.h5', 'table', append=False, data_columns=True)
#del sbe3